In [ ]:
#TODO: Schedule to run daily

In [46]:
import pandas as pd
import requests
import json
import pyodbc
import urllib
import dpath.util
from datetime import date, timedelta
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [52]:
yesterday = date.today() - timedelta(days = 1)
date = yesterday.strftime('%Y-%m-%d')

In [55]:
#Get data from USGS using JSON format
#Full URL for example
#url = "HTTP://waterservices.usgs.gov/nwis/iv/?format=json&sites=03432350&startDT=2022-09-02T00:00-0500&endDT=2022-09-06T23:59-0500&parameterCd=00060,00065&siteStatus=all"

url = "HTTP://waterservices.usgs.gov/nwis/iv/?format=json&sites=03432350&startDT=" + date + "T00:00-0500&endDT=" + date + "T23:59-0500&parameterCd=00060,00065&siteStatus=all"
response = requests.get(url)

#Convert data to dictionaries
JSONdict = json.loads(response.content)

In [56]:
#Extract values of flowrates and gaugeheights from JSON
FlowRates = pd.json_normalize(dpath.util.get(JSONdict, 'value/timeSeries/0/values/0/value'))
GaugeHeights = pd.json_normalize(dpath.util.get(JSONdict, 'value/timeSeries/1/values/0/value'))

In [27]:
#Merge gauge height and flow rates, change column names
FinalValues = pd.merge(FlowRates, GaugeHeights, on = 'dateTime', how = 'outer')
FinalValues.rename(columns = {'value_x' : 'FlowRate', 'qualifiers_x': 'Qualifier_FlowRate', 'value_y' : 'GaugeHeight', 'qualifiers_y' : 'Qualifier_GaugeHeight', 'dateTime' : 'Datetime'}, inplace = True)

In [28]:
#Reformat datetimes
FinalValues['Datetime'] = FinalValues['Datetime'].str.replace('T', ' ')
FinalValues['Datetime'] = FinalValues['Datetime'].str[:-10]

In [29]:
#Extract strings from qualifier objects
FinalValues['Qualifier_FlowRate'] = FinalValues['Qualifier_FlowRate'].str[0]
FinalValues['Qualifier_GaugeHeight'] = FinalValues['Qualifier_GaugeHeight'].str[0]

In [30]:
#Replace NA with 0
FinalValues['FlowRate'].fillna(0, inplace = True)
FinalValues['Qualifier_FlowRate'].fillna('', inplace = True)

In [31]:
#Ensure numeric data are numeric
FinalValues['GaugeHeight'] = pd.to_numeric(FinalValues['GaugeHeight'])
FinalValues['FlowRate'] = pd.to_numeric(FinalValues['FlowRate'])

In [32]:

#FinalValues['Qualifier_FlowRate'] = FinalValues['Qualifier_FlowRate'].astype(str)
#FinalValues['Qualifier_GaugeHeight'] = FinalValues['Qualifier_GaugeHeight'].astype(str)

In [33]:

connection = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-1SCLIO6;'
                      'Database=RiverHeightPrediction;'
                      'Trusted_Connection=yes;')

cursor = connection.cursor()

for index,row in FinalValues.iterrows():
    cursor.execute("INSERT INTO dbo.RiverData([Datetime], [GaugeHeight], [FlowRate], [Qualifier_GaugeHeight], [Qualifier_FlowRate]) values (?,?,?,?,?)", 
                                          
                        row['Datetime'],
                        row['GaugeHeight'],
                        row['FlowRate'],
                        row['Qualifier_GaugeHeight'],
                        row['Qualifier_FlowRate'])
    
    
    
    connection.commit()
cursor.close()
connection.close()